# SatNOGS Processor

This notebook showcases how to use SatNOGS (Satellite Networked Open Ground Station) with TorchSig. This downloads 1 observations from satnogs and creates a spectrogram and PNGs file. Then allows the user to draw bounding boxes on the spectrogram, the spectrogram is processed into Torchsig to make predictions. The torchsig predictions are given back to the user for further refinement via sigmf.

Note:  This notebook must be ran in Jupyter notebook due to Yolo annotation tool and may not work properly in VS code 
---

In [ ]:
!pip install librosa

In [ ]:
""" This notebook grabs signals from Satnogs and then converts it to sigmf and then runs it through Torchsig"""
import os
import tempfile
import subprocess
import math
from datetime import datetime, timedelta
import requests
from bs4 import BeautifulSoup
import numpy as np
from ultralytics import YOLO
import librosa
import cv2
from scipy import signal
import torchaudio
import torch
import matplotlib.pyplot as plt
from jupyter_bbox_widget import BBoxWidget
from sigmf import SigMFFile, sigmffile
from torchsig.image_datasets.datasets.yolo_datasets import YOLOFileDataset
from torchsig.image_datasets.annotation_tools.yolo_annotation_tool import yolo_annotator

In [ ]:
# Get the current date
now = datetime.today()
yesterday = now - timedelta(days=1)
date =  yesterday.strftime("%Y-%m-%d")
PAGENUM = 1
NEXTPAGE = 1
waterfall_links = []
BASE_SATNOGS_URL = 'https://network.satnogs.org/observations/'
x = 1

In [ ]:
while NEXTPAGE == 1 and x == 1:
    URELDATE = f'https://network.satnogs.org/observations/?norad=&start={date}'
    URLRES = '+18%3A55&end=&observer=&station=&results=w1&results=a1&results=d1&rated=rw1&'
    MAINURL = URELDATE + URLRES + f'transmitter_mode=&transmitter_uuid=&page={PAGENUM}'

    print(f"\nCurrently on page: {PAGENUM}")
    print(f"Current item count: {x}")

    try:
        mainresponse = requests.get(MAINURL, timeout=10)

        if mainresponse.status_code == 200:
            # Parse the HTML content of the main page
            mainsoup = BeautifulSoup(mainresponse.text, 'html.parser')
            disable_next = mainsoup.select_one('#page-selector > ul > li.page-item.disabled')
            if disable_next and PAGENUM > 1:
                NEXTPAGE = 0
                print("On the last page, setting NEXTPAGE to 0.")

            # Find all elements with the class "badge badge-good" (these contain observation IDs)
            GoodStatus = mainsoup.find_all(class_='badge badge-good')
            for item in GoodStatus:
                if x != 1:
                    print(f"Reached item limit of {x}. Stopping inner loop.")
                    break
                observation_id = item.text.strip()
                ITEMURL = f'https://network.satnogs.org/observations/{observation_id}'

                print(f"  Processing observation ID: {observation_id}")
                try:
                    itemresponse = requests.get(ITEMURL, timeout=10)
                    if itemresponse.status_code == 200:
                        itemsoup = BeautifulSoup(itemresponse.text, 'html.parser')
                        title = itemsoup.title
                        if title:
                            print(f"    Page Title: {title.text.strip()}")
                        else:
                            print(f"    Warning: Could not find page title for {observation_id}.")
                        data_element = itemsoup.find('a', href=lambda x_href: x_href and
                                                                    'satnogs' in x_href and
                                                                    x_href.endswith('.ogg'))
                        if data_element:
                            waterfall_link = data_element['href']
                            waterfall_links.append(waterfall_link)
                            x += 1
                            print(f"    Successfully found and saved audio link. Total items: {x}")
                        else:
                            print(f"    Audio link ('.ogg') not found for observation {observation_id}.")
                    else:
                        print(f"    Failed to retrieve data for observation {observation_id}. "
                                f"Status code: {itemresponse.status_code}")
                except requests.exceptions.RequestException as e:
                    print(f"    An error occurred while requesting {ITEMURL}: {e}")
                except Exception as e:
                    print(f"    An unexpected error occurred for observation {observation_id}: {e}")
            if x != 1:
                print(f"Overall item limit of {x} reached. Setting NEXTPAGE to 0 to stop main loop.")
                NEXTPAGE = 0 #

        else:
            # If the main page request failed
            print(f"Failed to retrieve main page. Status code: {mainresponse.status_code}")
            NEXTPAGE = 0 # Stop the loop if main page fails
    except requests.exceptions.RequestException as e:
        # Catch network errors for the main page request
        print(f"Network error occurred while fetching main page {MAINURL}: {e}")
        NEXTPAGE = 0 # Stop the loop on network error
    # Increment to the next page number for the next iteration of the while loop
    PAGENUM += 1
print(waterfall_links)

In [ ]:
def torchaudioused(torchchunk):
    """
    Determines the normalized spectrogram
    """

    torchnfft = 1024
    specgram = torchaudio.transforms.Spectrogram(
        n_fft=torchnfft*2,
        win_length=torchnfft*2,
        hop_length=torchnfft*2,
        window_fn=torch.blackman_window,
        normalized=False,
        center=False,
        onesided=True,
        power=True,
    )
    norm = lambda x: torch.linalg.norm(
        x,
        ord=float("inf"),
            keepdim=True,
    )
    torchx = specgram(torch.from_numpy(torchchunk))
    torchx = torchx[:-1]
    torchx = torchx * (1 / norm(torchx.flatten()))
    torchx = torchx.flipud()
    torchx = 10*np.log10(torchx.numpy()+1e-12)
    return torchx

In [ ]:
def baseband_downsample_real_to_real(input_data,r_cfreq,signal_freq,israte,osrate):
    """
    reduces the data and sample rate due to a constraint
    """
    if torch.cuda.is_available() is True:
        with torch.cuda.device('cuda:0'):
            input_data.to('cuda:0')
            inx = torch.linspace(0,len(input_data)-1,len(input_data),
                               dtype=torch.complex64,device='cuda:0')
            fshift = (r_cfreq-signal_freq)/(israte*1.0)
            fv = math.e ** (1j*2* math.pi *fshift*inx)
            fv.to('cuda:0')
            input_data = input_data * fv
            num_samples_in = len(input_data)
            # print(len(input_data))
            num_samples = int(np.ceil(num_samples_in/(israte/osrate)))
            # print (int((num_samples*israte)))
            osrate_new = int((num_samples*israte)/num_samples_in)
            down_factor = int(israte/osrate_new)
            transform = torchaudio.transforms.Resample(int(israte/osrate_new),
                                                       1,dtype=torch.float32).to('cuda:0')
            #test = transform(input_data.real) + 1j*transform(input_data.imag)
            test = transform(input_data.real)
            return test,(israte/down_factor)

In [ ]:
def Save_PNG(directory,sample_rate,pngname,NFFT,x):
    """
    Saves an image
    """
    if sample_rate > 0:
        sampleRate.update({pngname: sample_rate})
        x = cv2.resize(x, (NFFT, NFFT), interpolation=cv2.INTER_LINEAR)
    #convert to 3 channel grayscale black hot image using opencv
    pngimg_new = np.zeros((NFFT, NFFT, 3),dtype=np.float32)
    pngimg_new = cv2.normalize(x, pngimg_new, 0, 255, cv2.NORM_MINMAX)
    pngimg_new = cv2.cvtColor(pngimg_new.astype(np.uint8), cv2.COLOR_GRAY2BGR)
    pngimg_new = cv2.bitwise_not(pngimg_new)
    # Create the full path for the new file
    imgtext = "image.png"
    pngimgname =  pngname + imgtext
    pngoutput_path = os.path.join(directory,pngimgname)  # Change extension as needed
    print(pngoutput_path)
    if cv2.imwrite(pngoutput_path, pngimg_new):
        print(f"Image saved successfully: {pngoutput_path}")
    else:
        print("Failed to save image.")

In [ ]:
#sets constants nfft, Id (current ogg in audio_links, starts at 1)
NFFT = 1024
NFFTNFFT = NFFT * NFFT * 2
ID = 0
#determines if the user wants to refine the data,
#if they do, It will require further user input.
    #Otherwise will run through the entire file automatically
edit = input("Would you like to manually refine the data Y/N").upper()
# Defines the directory in which final images will be saved in
DIRECTORY = 'signals/dataset'
if not os.path.exists(DIRECTORY):
    os.makedirs(DIRECTORY)

refdir = os.path.join(DIRECTORY, 'refined')
if not os.path.exists(refdir):
    os.makedirs(refdir)

undir = os.path.join(DIRECTORY, 'unrefined')
if not os.path.exists(undir):
    os.makedirs(undir)

#iterates through the entire audio_link file
#2
sampleRate = {}
for link in waterfall_links:
    response = requests.get(link)

    if response.status_code == 200:
        # Create a temporary ogg file in the system's temp directory
        with tempfile.NamedTemporaryFile(delete=False, suffix='.ogg') as temp_file:
            temp_file.write(response.content)
            fileName = temp_file.name
            #converts the ogg data into a numpy array (data) and
                #determines the sample_rate for later use.
            try:
                data, sample_rate = librosa.load(temp_file.name,sr=None)
            except:
                break
        # #Increments id for each new data and creates chunks from the data
        print(data)
        ID = ID +1
        # print(type(data))
        print(data[:NFFT*NFFT])
        # chunk = data[:NFFT*NFFT]
        # print(chunk)
        lenSamples = len(data)
        chunk_count_rounded = int(lenSamples/NFFTNFFT)
        print(int(lenSamples))
        data = data[:(chunk_count_rounded*NFFTNFFT)]
        data = data.reshape(chunk_count_rounded,NFFTNFFT)
        print(data.shape)
        # Goes through the chunks of the data file and converts them into usable pngs for yolo.
        for i in range(chunk_count_rounded):
            IDNAME = str(ID)+","+str(i)
            # print(data[i].shape)
            chunk = data[i]
            #print(chunk.shape)
            #print(i)
            chunkx = torchaudioused(chunk)
            #3
            #If the user has requested to refine the data,
                #it will pass them to the refine function.
            sampleRate.update({IDNAME: sample_rate})
            if edit in ('Y', 'YES'):
                Save_PNG(undir,0,IDNAME,NFFT,chunkx)
            else:
                Save_PNG(refdir,0,IDNAME,NFFT,chunkx)
                print("no refinement")

In [ ]:
def tosigmf (signame,sigidname, manual,sigedit):
    """
      converts to SIGMF file
    """
    sigauthor = "jane.doe@domain.org"
    sigdes = "ogg real file."
    sigver = "1.0.0"
    sigpngname = signame.replace(".txt", "")
    sigpng_path = os.path.join(refdir, sigpngname)
    # Define basic metadata
    if signame !='.ipynb_checkpoints':
        while sigedit in ('Y', 'YES'):
            sigfield = input("What field would you like to edit? author, description, or version")
            match sigfield:
                case "author":
                    sigauthor = input("What would you like to set as the author of this file?")
                case "description":
                    sigdes = input("What would you like to set as the description of this file?")
                case "version":
                    sigver = input("What would you like to set as the version of this file?")
            sigedit = input("Would you like to edit another field? Y/N")
        sigsample_rate = sampleRate[sigidname]
        sigmeta = SigMFFile(
            global_info = {
                SigMFFile.DATATYPE_KEY: 'cf64_le',
                SigMFFile.SAMPLE_RATE_KEY: sigsample_rate,
                SigMFFile.AUTHOR_KEY: sigauthor,
                SigMFFile.DESCRIPTION_KEY: sigdes,
                SigMFFile.VERSION_KEY: sigver,
            }
        )
        if not manual:
            for siglabel in labels:
                i = 0
                sx, sigcx, sigcy, sigw, _ = siglabel
                sigcx = sigcx * width
                sigsx = sigcx - ((sigw * width) // 2)
                sigex = sigcx + ((sigw * width) // 2)
                sigcy = (sigcy * 2) * int(sigsample_rate /2)

                if len(labels) == 1:
                    #crops image to only where the signal has been identified, which will prevent the sigmf file from being created
                    refdata, refsample_rate = baseband_downsample_real_to_real(torch.from_numpy(chunk[(int(sigsx)*2048):(int(sigex)*2048)]).to('cuda:0'),
                                                                               int(int(sample_rate)/4),int(sigcy),48000,1600)
                    #overwrites data in annotation dataset
                    refdata = refdata[:(chunk_count_rounded*NFFTNFFT)]
                    refdata = refdata.cpu().numpy()
                    x = torchaudioused(refdata)
                    print("x", x)
                    Save_PNG(refdir,refsample_rate,sigidname,NFFT,x)

                sigmeta.add_capture(sx, metadata={
                    "core:sample_start":sigsx,
                    "core:sample_count":1,
                    "core:frequency": sigcy,
                    "core:comment": i,
                })
                sigmeta.validate()
                output_path = os.path.join(refdir, metaname)
                sigmeta.tofile(output_path)
                i = i + 1

        else:
            sigsx = labels[0]
            sigex = labels[1]
            sigcy = labels[2]
            try:
                #crops image to only where the signal has been identified, which will prevent the sigmf file from being created
                refdata, refsample_rate = baseband_downsample_real_to_real(torch.from_numpy(chunk[(int(sigsx)*2048):(int(sigex)*2048)]).to('cuda:0'),
                                                                           int(int(sample_rate)/4),int(sigcy),48000,1600)
                # #overwrites data in annotation dataset
                data = refdata[:(chunk_count_rounded*NFFTNFFT)]
                x = torchaudioused(data)

                Save_PNG(refdir,refsample_rate,sigidname,NFFT,x)
                #6
                sigmeta.add_capture(sx, metadata={
                    "core:sample_start":sigsx,
                    "core:sample_count":1,
                    "core:frequency": sigcy,
                    "core:comment": 1,
                })
                sigmeta.validate()
                output_path = os.path.join(refdir, metaname + ".sigmf-meta")
                # print(output_path)
                sigmeta.tofile(output_path)
            except:
                print("no meta saved")

In [ ]:
#data that needs refrainment via manual input
if edit == "Y":
    for file in os.listdir(undir):
        filename = os.fsdecode(file)
        if filename.endswith(".png"):
            name = filename.replace(".png","")
            idname = name.replace("image_name", "")
            metaname = name + "sigmf-meta"
            imagepath = os.path.join(undir,filename)
            # print(imagepath)
            # Load the image
            image = cv2.imread(imagepath)
            # Adjust the brightness and contrast
            # Adjusts the brightness by adding 10 to each pixel value
            BRIGHTNESS = 10
            # Adjusts the contrast by scaling the pixel values by 2.3
            CONTRAST = 2.3
            image2 = cv2.addWeighted(image, CONTRAST, np.zeros(image.shape, image.dtype), 0, BRIGHTNESS)

            figure = plt.figure(figsize=(30,30))
            ax = plt.subplot()
            figure.suptitle("with axis 48K real")
            ax.imshow(image)
            ax.set_xlabel("Duration:")
            ax.set_ylabel("Bandwidth:")
            ax.set_yticks(list(np.linspace(1023,0,100)),list(np.linspace(0,int(sample_rate/2),100)))
            plt.show()
            mansx = input("Enter start duration")
            manex = input("Enter end duration")
            mancy = input("Enter frequency")
            manlabels = [mansx,manex,mancy]
            tosigmf (name,idname, True)

In [ ]:
#data that needs refrainment via drawling
from torchsig.image_datasets.annotation_tools.yolo_annotation_tool import yolo_annotator
if edit == "Y":
    UNLAB_DIR = "signals/dataset/unrefined/" # directory of images to be annotated
else:
    UNLAB_DIR = "signals/dataset/refined/"
NEWYOLODT = "signals/annotated_dataset/" # directory to save annotated yolo data
yolo_annotator(UNLAB_DIR,NEWYOLODT)

In [ ]:
#data that needs refrainment via drawling
from torchsig.image_datasets.datasets.yolo_datasets import YOLOFileDataset
plt.rcParams["figure.figsize"] = (50,50)
yds1 = YOLOFileDataset(NEWYOLODT)
yds1.root_filepath, os.listdir(yds1.root_filepath + "labels")

In [ ]:
print("current defaults for all files: Author: jane.doe@domain.org Description: ogg real file. Version Key: 1.0.0") 
sigedit = input("would you like to edit the default fields for all files? Y/N").upper()

for x in range (len(yds1)):
    image= yds1[x].img
    height, width = image.shape[:2]
    labels = yds1[x].labels
    name = os.listdir(yds1.root_filepath + "labels")[x]
    name = name.replace(".png","")
    idname = name.replace("image", "")
    idname = idname.replace(".txt", "")
    metaname = name.replace(".txt", ".sigmf-meta")
    meta_path = os.path.join(refdir, metaname)
    # print(name + "-" + idname + "-" + metaname)
    #metadata
    tosigmf(name,idname, False,sigedit)

In [ ]:
class UniqueListOfLists:
    """
    Make sure that the list only contains unique values
    """
    def __init__(self):
        self._lists = []

    def add(self, new_list):
        """
        Adds unique values to the list
        """
        if new_list not in self._lists:
            self._lists.append(new_list)

    def __iter__(self):
        return iter(self._lists)

    def __repr__(self):
        return f"UniqueListOfLists({self._lists})"

In [ ]:
%%bash
destination_path=detect.pt
download_url=https://bucket.ltsnet.net/torchsig/models/detect.pt

curl -L -o "$destination_path" "$download_url"

if [ $? -eq 0 ]; then
    echo "Download completed successfully."
else
    echo "Download failed."
fi

destination_path=11s.pt
download_url=https://bucket.ltsnet.net/torchsig/models/11s.pt

curl -L -o "$destination_path" "$download_url"

if [ $? -eq 0 ]; then
    echo "Download completed successfully."
else
    echo "Download failed."
fi

destination_path=xcit.ckpt
download_url=https://bucket.ltsnet.net/torchsig/models/xcit.ckpt

curl -L -o "$destination_path" "$download_url"

if [ $? -eq 0 ]; then
    echo "Download completed successfully."
else
    echo "Download failed."
fi

In [ ]:
#passing sigmf files to model for predictions
model = YOLO('detect.pt')
result = model.named_parameters
# Load a dataset
for file in os.listdir(refdir):
    filename = os.fsdecode(file)
    if filename.endswith("-meta"):
        output_path = os.path.join(refdir,filename)
        # print(output_path)

        signal = sigmffile.fromfile(output_path)
        samples = 1

        #removes any current annotations from file
        filetxt = filename.replace(".sigmf-meta", ".txt")
        file_path = os.path.join("signals/annotated_dataset/labels/",filetxt)
        # print(file_path)

        fileNamepng = filename.replace(".sigmf-meta", ".png")
        fileNamepng = fileNamepng.replace(".png.png", ".png")
        file_png_path = os.path.join("signals/dataset/refined/",fileNamepng)
        # print(file_png_path)
        labels = UniqueListOfLists()
        for x in range(len(signal.get_captures())):
            # Get some metadata and all annotations
            fc = signal.get_captures()[x]['core:frequency']
            sample_rate = signal.get_global_field(SigMFFile.SAMPLE_RATE_KEY)
            sample_count = signal.get_captures()[x][ "core:sample_count"]
            signal_duration = sample_count / sample_rate
            fs = sample_rate
            file_png_path = os.path.join("signals/dataset/refined/",fileNamepng)
            if cv2.haveImageReader(file_png_path):
                img_new = cv2.imread(file_png_path)
            else:
                continue
            result = model(img_new , imgsz=NFFT, save=False, augment=False,iou=0.1, max_det=300)
            plot_img = result[0].orig_img
            height, width = plot_img.shape[:2]
            print(height,width)
            Z = 0
            # print(result[0].boxes.xyxy)
            if result[0].boxes.xyxy != []:
                for Z, boxes_xyxy in enumerate(result[0].boxes.xyxy):
                    y = 0
                    box_xyxy = boxes_xyxy.cpu().numpy()
                    box_xywh = result[0].boxes.xywh[Z].cpu().numpy()
                    center_freq = (float(fs)/2.0)-(float(box_xywh[1]/NFFT)*float(fs))+fc
                    top_freq = (float(fs)/2.0)-((box_xyxy[1]/NFFT)*float(fs))+fc
                    bottom_freq = (float(fs)/2.0)-((box_xyxy[3]/NFFT)*float(fs))+fc
                    bandwidth = top_freq - bottom_freq
                    start_sample = int(box_xyxy[0])*int(NFFT)
                    end_sample = int(box_xyxy[2])*int(NFFT)
                    duration = end_sample - start_sample
                    signal.add_annotation(start_sample,duration, metadata = {
                        SigMFFile.FLO_KEY: bottom_freq,
                        SigMFFile.FHI_KEY: top_freq,
                        SigMFFile.COMMENT_KEY:str(y),
                    })
                    # print(box_xyxy[0])
                    cx = (box_xyxy[0] + box_xywh[2]//2)/width
                    cy = (box_xyxy[1] + box_xywh[3]//2)/height
                    # print(cx)
                    new_width = box_xywh[2]/width
                    # print(new_width)
                    new_height = box_xywh[3]/height
                    # print(new_height)
                    # print("end:" + str(cx))
                    print(y)
                    labels.add([cx,cy,new_width,new_height])
                    y = y + 1
                signal.tofile(output_path)
            with open(file_path, "w") as file:
                y = 0
                for line in labels:
                    file.write(str(y) + " "+ str(line[0]) + " "+ str(line[1]) + " "+str(line[2])+ " "+ str(line[3]) + "\n")
                    y = y + 1

In [ ]:
#creates dataset of all annotations images
plt.rcParams["figure.figsize"] = (50,50)
YOLODIR = "signals/annotated_dataset/" # directory to save annotated yolo data
yds1 = YOLOFileDataset(YOLODIR )
# type(yds1)

In [ ]:
try:
    yds1.root_filepath, os.listdir(yds1.root_filepath + "labels")
    yds1[0]
    # print(yds1[1].labels)
except:
    raise ValueError('No annotated images in directory, please either 1) re-run and annotate images or 2) if the tool is unavailable please run in jupyter lab')

In [ ]:
for x in range (len(yds1)):
    imagename = os.listdir(yds1.root_filepath + "images")[x]
    imagename =imagename[:-3] + "png"
    print(imagename)
    image= yds1[x].img
    boxclass = []
    labels = yds1[x].labels
    image = 1 - image.numpy()
    if labels != [[]]:
        # font
        font = cv2.FONT_HERSHEY_SIMPLEX
        # fontScale
        FONTSCALE = 1
        # Blue color in BGR
        color = (0, 255, 0)
        image = (np.stack([image[0,:,:]]*3).transpose(1,2,0)*255).astype(np.uint8)
        for label in labels:
            cid, cx, cy, w, h = label
            print(label)
            img_h, img_w = image.shape[:2]
            x1 = int((cx - w/2)*img_w)
            x2 = int((cx + w/2)*img_w)
            y1 = int((cy - h/2)*img_h)
            y2 = int((cy + h/2)*img_h)
            image = cv2.rectangle(image.copy(), (x1, y1), (x2, y2), color=(255,0,0), thickness=1)
            # org
            org = (x1, y1)
            # Using cv2.putText() method
            image = cv2.putText(image.copy(), str(cid) , org, font, FONTSCALE, color)
            print(cid)

        plt.imshow(image)
        outimg_path = os.path.join("signals/dataset/refined", imagename)  # Change extension as needed
        if cv2.imwrite(outimg_path, image):
            print(f"Image saved successfully: {outimg_path}")
        else:
            print("Failed to save image.")


In [ ]:
#gets the users input on what labels they would like to change
changeimagename = input("What image would you like to update EX:1,14")
edit = input("What labels would you like to edit")


In [ ]:
#creates output path for img for widget
changeimagename = changeimagename.replace(".", ",")
output_path = os.path.join("signals/dataset/refined/", (changeimagename  + "image.png"))
print(output_path)


In [ ]:
#creates widget information
widget = BBoxWidget(
    image= output_path,
    classes= ['signal'],
)


In [ ]:
@widget.on_submit
def submit():
    """
    sets up widget
    """
    widget.close()


In [ ]:
widget

In [ ]:
widget.bboxes

In [ ]:
# Load a sigmf dataset
filesigmf = output_path[:-4]

print("sigmf", filesigmf)
signal = sigmffile.fromfile(filesigmf)
print(signal)
#initialize other global variables
img = plt.imread(output_path)
height, width = img.shape[:2]
edits = edit.split(",")
nlabels = []
labels = []
annotations = []
ydsimage= yds1[0].img

output_path = output_path.replace("image_name", "")
filemeta =  output_path.replace(".png", ".sigmf-meta")
filetxt = output_path.replace(".png", "image.txt")
filetxt = filetxt.replace("signals/dataset/refined/", "")
filetxt = filetxt.replace("imageimage", "image")
filename = os.path.join("signals/annotated_dataset/labels/" , changeimagename + "image.txt" )
#iterates through the new annotations
for box in widget.bboxes:
    # print(box)
    cx = (box['x'] + box['width']//2)/width
    cy = (box['y'] + box['height']//2)/height
    sx = cx - box['width']// 2
    ex = cx + box['width'] // 2
    new_width = box['width']/width
    new_height = box['height']/height
    duration = ex - sx
    fs = sample_rate
    top_freq = (float(fs)/2.0)-((box['y']/NFFT)*float(fs))+cy
    bottom_freq = (float(fs)/2.0)-((box['height']/NFFT)*float(fs))+cy
    # Retrieve annotations at index z
    annotations += [[cy, bottom_freq,top_freq,duration]]
    labels += [[cx, cy, new_width, new_height]]
# print(labels)


#updates meta and yds files
print(filetxt)
for y in range(len(edits)):
    print(y)
    an = signal.get_annotations()
    # print(an)
    if an[y]['core:comment'] == edits[y]:
        an[y]['core:sample_start'] == annotations[y][0]
        an[y]['core:freq_lower_edge'] = annotations[y][1]
        an[y]['core:freq_upper_edge'] = annotations[y][2]
        an[y]['core:sample_count'] =  annotations[y][3]
        signal.validate()
        # print(filemeta)
        signal.tofile(filemeta)
    # print(filename)
    with open(filename, "r") as file:
        for line in file:
            item = line.split()  # Automatically splits on whitespace
            if item[0] == edit[y]:  # Check if the first element matches edit[y]
                # Create the new label list
                nlabel = [item[0], labels[y][0], labels[y][1],labels[y][2], labels[y][3]]
                if nlabel not in nlabels:
                    nlabels.append(nlabel)
            else:
                nlabels.append(item)  # Append the original item if no match
print(nlabels)


#updates annotates_dataset txt labels
with open(filename, "w") as file:
    for line in nlabels:
        file.write(str(line[0]) + " "+ str(line[1]) + " "+ str(line[2]) + " "+str(line[3])+ " "+ str(line[4]) + "\n")

# saves and displays updated photo
INDEX = 0
for x in range (len(yds1)):
    imgname = os.listdir(yds1.root_filepath + "images")[x]
    imgname =imgname[:-3] + "png"
    print(imagename)
    if imgname == imagename:
        INDEX = x
print(INDEX)

image= yds1[INDEX].img
boxclass = []
labels = nlabels
image = 1 - image.numpy()

# font
FONT = cv2.FONT_HERSHEY_SIMPLEX
# fontScale
FONTSCALE = 1
# Blue color in BGR
color = (0, 255, 0)
image = (np.stack([image[0,:,:]]*3).transpose(1,2,0)*255).astype(np.uint8)
for label in labels:
    cid, cx, cy, w, h = label
    img_h, img_w = image.shape[:2]
    x1 = int((float(cx) - float(w)/2)*img_w)
    x2 = int((float(cx) + float(w)/2)*img_w)
    y1 = int((float(cy) - float(h)/2)*img_h)
    y2 = int((float(cy) + float(h)/2)*img_h)
    image = cv2.rectangle(image.copy(), (x1, y1), (x2, y2), color=(255,0,0), thickness=1)
    # org
    org = (x1, y1)
    # Using cv2.putText() method
    image = cv2.putText(image.copy(), str(cid) , org, FONT, FONTSCALE, color)

plt.imshow(image)
if cv2.imwrite(output_path, image):
    print(f"Image saved successfully: {output_path}")
else:
    print("Failed to save image.")